In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['HF_HOME'] = '/data/hf_cache'
import pandas as pd
import requests
import json
from tqdm import tqdm
import datetime as dt
from time import sleep
import seaborn as sns
import matplotlib.pyplot as plt
from bertopic import BERTopic
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

/data/conda/envs/jelena/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/data/conda/envs/jelena/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/data/conda/envs/jelena/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecatio

In [2]:
GROUP = 1
CONFIG = 1

In [ ]:
model_1 = BERTopic.load(f"../models/group_{GROUP}/model_group_1_dbcv_0_44", embedding_model="sentence-transformers/allenai-specter")
model_2 = BERTopic.load(f"../models/group_{GROUP}/random_LLM_model_group_1_dbcv_0_52", embedding_model="Salesforce/SFR-Embedding-2_R")

In [5]:
topics_model_1 = model_1.get_topic_info()
topics_model_2 = model_2.get_topic_info()

In [6]:
keywords_model_1 = [model_1.get_topic(topic)[0:10] for topic in topics_model_1.Topic]
keywords_model_2 = [model_2.get_topic(topic)[0:10] for topic in topics_model_2.Topic]

In [7]:
topics_1_strings = [' '.join([kw[0] for kw in keywords]) for keywords in keywords_model_1]
topics_2_strings = [' '.join([kw[0] for kw in keywords]) for keywords in keywords_model_2]

In [8]:
vectorizer = CountVectorizer().fit(topics_1_strings + topics_2_strings)
topic_1_vectors = vectorizer.transform(topics_1_strings)
topic_2_vectors = vectorizer.transform(topics_2_strings)

In [9]:
similarity_matrix = cosine_similarity(topic_1_vectors, topic_2_vectors)

In [18]:
non_macth_similarity_threshold = 0.2
model_1_non_matches = similarity_matrix.max(axis=1) <= non_macth_similarity_threshold
model_1_non_found_count = model_1_non_matches.sum()
model_2_non_matches = similarity_matrix.max(axis=0) <= non_macth_similarity_threshold
model_2_non_found_count = model_2_non_matches.sum()

In [19]:
macth_similarity_threshold = 0.4
model_1_matches = similarity_matrix.max(axis=1) >= macth_similarity_threshold
model_1_found_count = model_1_matches.sum()
model_2_matches = similarity_matrix.max(axis=0) >= macth_similarity_threshold
model_2_found_count = model_2_matches.sum()

In [ ]:
print(f"Topics from Model 1 not found in Model 2: {model_1_non_found_count} / {len(keywords_model_1)}")
print(f"Topics from Model 2 not found in Model 1: {model_2_non_found_count} / {len(keywords_model_2)}")
model_1_non_found_percentage = (model_1_non_found_count / len(keywords_model_1)) * 100
model_2_non_found_percentage = (model_2_non_found_count / len(keywords_model_2)) * 100

print(f"Topics from Model 1 not found in Model 2: {model_1_non_found_count} / {len(keywords_model_1)} "
      f"({model_1_non_found_percentage:.2f}%)")

print(f"Topics from Model 2 not found in Model 1: {model_2_non_found_count} / {len(keywords_model_2)} "
      f"({model_2_non_found_percentage:.2f}%)")

In [ ]:
print(f"Topics from Model 1 found in Model 2: {model_1_found_count} / {len(keywords_model_1)}")
print(f"Topics from Model 2 found in Model 1: {model_2_found_count} / {len(keywords_model_2)}")
model_1_found_percentage = (model_1_found_count / len(keywords_model_1)) * 100
model_2_found_percentage = (model_2_found_count / len(keywords_model_2)) * 100

print(f"Topics from Model 1 found in Model 2: {model_1_found_count} / {len(keywords_model_1)} "
      f"({model_1_found_percentage:.2f}%)")

print(f"Topics from Model 2 found in Model 1: {model_2_found_count} / {len(keywords_model_2)} "
      f"({model_2_found_percentage:.2f}%)")

In [ ]:
# matched_topics_df_model_1 = pd.DataFrame({
#     'Model 1 Topic': np.where(model_1_matches)[0],
#     'Model 1 Keywords': [topics_1_strings[i] for i in np.where(model_1_matches)[0]],
#     'Model 2 Topic': similarity_matrix.argmax(axis=1)[model_1_matches],
#     'Model 2 Keywords': [topics_2_strings[i] for i in similarity_matrix.argmax(axis=1)[model_1_matches]],
#     'Cosine Similarity': similarity_matrix.max(axis=1)[model_1_matches]
# })

# matched_topics_df_model_2 = pd.DataFrame({
#     'Model 2 Topic': np.where(model_2_matches)[0],
#     'Model 2 Keywords': [topics_2_strings[i] for i in np.where(model_2_matches)[0]],
#     'Model 1 Topic': similarity_matrix.argmax(axis=0)[model_2_matches],
#     'Model 1 Keywords': [topics_1_strings[i] for i in similarity_matrix.argmax(axis=0)[model_2_matches]],
#     'Cosine Similarity': similarity_matrix.max(axis=0)[model_2_matches]
# })

# print("\nMatched Topics between Model 1 and Model 2:")
# print(matched_topics_df_model_2)

In [ ]:
# matched_topics_df_model_2.to_csv('macthing_topics_model_2.csv')